<h1>Testing the Monty Hall Problem</h1>

The <a href="https://en.wikipedia.org/wiki/Monty_Hall_problem">Monty Hall Problem</a> is a classic thought experiment and probability puzzle. It was first published in 1975 by statistician Steve Selvin in the scientific journal <i>American Statician</i>, and rose to popularity in 1990 when published in <i>Parade</i> magazine.

The Wikipedia article does a fantastic job explaining the rules, assumptions, and has many helpful ways of thinking about this unintuitive puzzle. I reccomend it for further reading for mathematical proofs and other explorations that are out of the scope of this exercise. Let's take a look at the basic rules and assumtions. The <i>Parade</i> version of the problem is as follows:

  <i>Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?" Is it to your advantage to switch your choice?</i>
  
When posed this problem, most people will respond that it doesn't matter if you switch or not, since there are two closed doors your chances 50/50. However, switching actually DOUBLES your chance of winning from 1/3 to 2/3, as correctly answered by <i>Parade</i> columnist Marylin vos Savant. This answer is so unintuitive that <i>Parade</i> received approximately 10,000 letters telling them that their answer was wrong. As I mentioned, the Wikipedia article has many tools and methods to help you understand why it is to your advantage to switch. However, I'm here to prove it experimentally through simulating the game 1,000 each switching choices, and not switching.

To start, we'll need to import the random module to help us generate our doors at random, and to help our virtual contestant and host make decisions:

In [1197]:
import random

Now, let's define a Door class. Each door will have a name (Door 1, Door 2, and Door 3), something behind it, and be initialized as "closed", represented as a boolean value. Here I've added a method to "open" the door, setting "is_closed" to "False" and adding what is in the "behind" attribute to the name to simulate the contestant's experience. 

In [1198]:
class Door:
    def __init__(self, name, behind):
        self.behind = behind
        self.name = name
        self.is_closed = True
    
    def __repr__(self):
        return self.name
    
    def open_door(self):
        if self.is_closed:
            self.name = self.name + ': ' + self.behind
            self.is_closed = False       

Now, let's define a function to generate doors based on a list of possible outcomes and assign each outcome randomly to a door. We copy the list to preserve the original (we'll use it again later in our code), so that we can remove each outcome as we assign it to a door, making sure it is only used once. Once complete, this function returns a list of Door objects the same length as our list of possible outcomes. For our purposes here, it will always be 3 items long.

In [1199]:
def generate_random_doors(possible_outcomes):
    outcomes = possible_outcomes[:]
    list_of_doors = []
    for i in range(len(outcomes)):
        rand_index = random.randint(0, (len(outcomes)-1))
        list_of_doors.append(Door('Door '+str(i+1), outcomes[rand_index]))
        outcomes.pop(rand_index)
    return list_of_doors

Now we'll define our list of possible outcomes, or what's behind each door we want to generate:

In [1200]:
possible_outcomes = ['car', 'goat', 'goat']    

Great. Now let's test our function with our established list of outcomes and print the result:

In [1201]:
test_doors = generate_random_doors(possible_outcomes)
print(test_doors)

[Door 1, Door 2, Door 3]


To make sure our open_door method is working correctly, let's iterate through our list and open each door, printing the result again at the end.

In [1202]:
for door in test_doors:
    door.open_door()
print(test_doors)

[Door 1: goat, Door 2: goat, Door 3: car]


Everything seems to be working so far. We have a way to randomly assign our outcomes to closed doors, and opening them allows us to easily see what's behind them. Now, let's create a Contestant class, with attributes that hold their "choice", a method to make that choice randomly from a list of closed doors, and a boolean value for whether they will switch their choice when given the option. 

In [1203]:
class Contestant:
    def __init__(self, always_switches):
        self.choice = None
        self.always_switches = always_switches
        
    def make_choice(self, choices):
        self.choice = choices[random.randint(0, (len(choices)-1))]
    
    def switch_choice(self, choices):
        for choice in choices:
            if choice != self.choice and choice.is_closed:
                self.choice = choice 
                break

Now let's make two instances of our Contestant class: one that always switches when given the choice, and one that never switches. We'll use these instances throughout the rest of our program.

In [1204]:
always_switches = Contestant(True)
never_switches = Contestant(False)

Our test doors are already open, so let's generate another set of test doors that are closed to make sure our "make_choice" and "switch_choice" methods work as expected.

In [1205]:
test_doors2 = generate_random_doors(possible_outcomes)
print(test_doors2)

[Door 1, Door 2, Door 3]


Have one of our contestants make a choice:

In [1206]:
always_switches.make_choice(test_doors2)
print(always_switches.choice)

Door 1


So now we have our list of doors, and our contestant has made their choice. We need to simulate the hosts actions in the next step. One of the keys to understanding this puzzle is that the host knows where the car is, and will not open the door with the car behind it. The host will also not open the door that our contestant has picked, and will only open one door. Let's write a quick function to simulate those actions by generating a random index from our list of doors and checking to see if the door at that index meets both of our conditions. If it doesn't, we recursively call the function until the random index <i>does</i> meets both our conditions, and open the door.

In [1207]:
def open_a_door(list_of_doors, contestant):
    rand_index = random.randint(0, (len(list_of_doors)-1))
    if list_of_doors[rand_index] != contestant.choice and list_of_doors[rand_index].behind != 'car':
            list_of_doors[rand_index].open_door()
    else:
        open_a_door(list_of_doors, contestant)
open_a_door(test_doors2, always_switches)
print(test_doors2)

[Door 1, Door 2: goat, Door 3]


Looking good! Our contestant made their selection, and our virtual host has opened one of the doors that they didn't choose to reveal a goat. Now, let's make sure our switch_choice method works as expected.

In [1208]:
always_switches.switch_choice(test_doors2)
print(always_switches.choice)

Door 3


Excellent! At this point we've simulated the entire gameplay, and whether the contestant wins or loses is determined by what's behind their door at this stage. Let's put all this functionality into one function that will return a boolean "True" if the contestant has chosen the door with the car behind it, and "False" otherwise.

In [1209]:
def play_game(contestant, possible_outcomes):
    new_doors = generate_random_doors(possible_outcomes)
    contestant.make_choice(new_doors)
    open_a_door(new_doors, contestant)
    if contestant.always_switches == True:
        contestant.switch_choice(new_doors)
    if contestant.choice.behind == 'car':
        return True
    else:
        return False 

Now we can simply use our play_game function to simulate the entire game, and have either True or False as a result if the contestant has won or lost. You can run the cell below a few times and watch the output change.

In [1210]:
play_game(always_switches, possible_outcomes)

True

Now, let's write a function that uses our play_game function, plays it some N number of times based on the input, logs the result to a list and returns that list.

In [1211]:
def play_game_n_times(n, contestant, possible_outcomes):
    result_list = []
    while len(result_list) < n:
        result_list.append(play_game(contestant, possible_outcomes))
    return result_list

Let's make sure this function is working correctly by printing out the result of playing the game 10 times:

In [1212]:
ten_results = play_game_n_times(10, always_switches, possible_outcomes)
for result in ten_results:
    print(result)

False
False
True
False
True
True
True
False
True
False


Fantastic! Now, let's simulate playing the game 1,000 times for both of our options: always switching and never switching, using both the Contestants we instantiated earlier.

In [1213]:
one_thousand_switches = play_game_n_times(1000, always_switches, possible_outcomes)
one_thousand_stays = play_game_n_times(1000, never_switches, possible_outcomes)

Each of these variables contains a list of 1,000 results of playing the game. Let's count them up and print out the results:

In [1214]:
switch_wins = 0
switch_losses = 0

stay_wins = 0
stay_losses = 0

for result in one_thousand_switches:
    if result:
        switch_wins += 1
    else:
        switch_losses += 1
        
for result in one_thousand_stays:
    if result:
        stay_wins += 1
    else:
        stay_losses += 1
        
print("Number of switch wins: {}".format(switch_wins))
print("Number of switch losses: {}".format(switch_losses))

print("Number of stay wins: {}".format(stay_wins))
print("Number of stay losses: {}".format(stay_losses))

Number of switch wins: 643
Number of switch losses: 357
Number of stay wins: 335
Number of stay losses: 665


At this point I think it's safe to say we've established a pretty clear pattern. Switching is clearly the winning strategy, winning about 2/3 the time, (666/1000) and not switching the contestant only wins about 1/3 of the time (333/1000).